# RNN 의사 코드

In [ ]:
state_t = 0 # 초기 상태

for input_t in input_sequence: # 각 시점에 해당하는 입력을 반복합니다.
    output_t = activation_func(input_t, state_t) # 입력과 은닉상태를 활성화 함수에 통과시킵니다.
    state_t = output_t # 출력값은 다음 시점을 위한 은닉 상태가 됩니다.

## sin 함수를 이용하여 데이터 만들기

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.sin(2 * np.pi * 0.05 + np.arange(0, 150)) + np.random.random(150)
plt.figure(figsize = (10, 5)) 
plt.plot(np.arange(0, 105), data[:105], 'bo-', color = 'blue', alpha = 0.2) # 학습용 데이터
plt.plot(np.arange(105, 150), data[105:], 'ro-', color = 'orange') # 테스트용 데이터

## 전처리 과정 수행하기

In [ ]:
def data_gen(data, n):
    x_train = []
    y_train = []
    
    for i in range(len(data)):
        x = data[i:(i + n)]
        if (i + n) < len(data):
            x_train.append(x)
            y_train.append(data[i + n])
        else:
            break
            
    return np.array(x_train), np.array(y_train)

n = 10

x_train, y_train = data_gen(data, n)
x_train = x_train.reshape(-1, n, 1)
y_train = y_train.reshape(-1, 1)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.25)
print(x_train.shape, y_train.shape) 
print(x_test.shape, y_test.shape)

## 데이터 형태 확인하기

In [ ]:
test_arr = np.arange(100)
a, b = data_gen(test_arr, 10)

for i in range(1, 4):
    print(a[i],'|', b[i])

## SimpleRNN을 사용하여 모델 구성하기

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Flatten, Dense
from tensorflow.keras.models import Sequential

model = Sequential()
# SimpleRNN 층을 첫 번째 층으로 사용하는 경우,
# 반드시 input_shape를 명시해주어야 합니다.
model.add(SimpleRNN(32, activation = 'tanh', input_shape = (n, 1)))
model.add(Dense(1, activation = 'linear'))

model.compile(optimizer = 'adam', loss = 'mse')
model.summary()

## 모델 학습시키기

In [ ]:
model.fit(x_train, y_train, epochs = 100, batch_size = 12)

## 예측 결과 그려보기

In [ ]:
pred = model.predict(x_test)

train_axis = np.arange(0, len(y_train))
pred_axis = np.arange(len(y_train), len(y_train) + len(pred))
train_shape= y_train.shape[0]
test_shape = y_test.shape[0]
pred_shape = pred.shape[0]

plt.figure(figsize=(15,5))
plt.plot(train_axis, y_train.reshape(train_shape,), 'o-')
plt.plot(pred_axis, pred.reshape(pred_shape,), 'o-', color='orange', label='pred')
plt.plot(pred_axis, y_test.reshape(test_shape,), 'o-', color='blue', alpha=0.2, label='ground-truth')
plt.legend()
plt.show()

## Embedding 층과 SimpleRNN 층을 사용하여 모델 구성하기

In [ ]:
from tensorflow.keras.datasets import imdb

num_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 500

pad_X_train = pad_sequences(X_train, maxlen=max_len)
pad_X_test = pad_sequences(X_test, maxlen=max_len)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding  

model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = 32))
# 새로운 인자 3개가 사용되었습니다.
model.add(SimpleRNN(32, return_sequences = True, dropout = 0.15, recurrent_dropout = 0.15))
model.add(SimpleRNN(32))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['acc'])

# model.summary()

history = model.fit(pad_X_train, y_train, 
                    batch_size = 32, epochs = 15, 
                    validation_split = 0.2)

## SimpleRNN 층의 출력값 변화 확인하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding  

model = Sequential()
model.add(Embedding(input_dim = 10000, output_dim = 32))
# 전체 상태 시퀀스를 반환하거나,
# 마지막 시점의 상태 시퀀스만 반환합니다.
model.add(SimpleRNN(32, return_sequences = True))
# model.add(SimpleRNN(32, return_sequences = False))
model.summary()

## 학습 과정 그리기

In [ ]:
import matplotlib.pyplot as plt

his_dict = history.history
loss = his_dict['loss']
val_loss = his_dict['val_loss'] 

epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize = (10, 5))

# 훈련 및 검증 손실 그리기
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(epochs, loss, color = 'blue', label = 'train_loss')
ax1.plot(epochs, val_loss, color = 'orange', label = 'val_loss')
ax1.set_title('train and val loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
ax1.legend()

acc = his_dict['acc']
val_acc = his_dict['val_acc']

# 훈련 및 검증 정확도 그리기
ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(epochs, acc, color = 'blue', label = 'train_acc')
ax2.plot(epochs, val_acc, color = 'orange', label = 'val_acc')
ax2.set_title('train and val acc')
ax2.set_xlabel('epochs')
ax2.set_ylabel('acc')
ax2.legend()

plt.show()